In [1]:
from random import randint, random
import collections
import os

import numpy as np
import math
import time
from collections import deque
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import sklearn.neighbors
import sklearn.metrics
import sklearn.mixture
import matplotlib.pyplot as plt
import PIL
from nvidia.dali.plugin.pytorch import DALIGenericIterator

from imageComponent import *
from remote_read_write import scp_read_wrapper, scp_write_wrapper
scp_Tload = scp_read_wrapper(T.load, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')
scp_Tsave = scp_write_wrapper(T.save, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')

dali_device = 3
device = 'cuda:2'
T.backends.cudnn.benchmark = True
T.backends.cudnn.deterministic = True
use_amp = False

/public/ly/anaconda3/envs/zyz/lib/python3.10/site-packages/nvidia/dali/backend.py:46: Warning: DALI support for Python 3.10 is experimental and some functionalities may not work.
  deprecation_warning("DALI support for Python 3.10 is experimental and some functionalities "


In [2]:
model = ViTCifar()
root = '/public/home/ly_1112103017/zyz/imageCL/ckpt/CLBadCifarViT'

In [3]:
trainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image2/train'
rawtrainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image2/train'
valfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image2/test'
size = (32, 32)
batch_size = 3125

In [4]:
rawtrainpipe = DALIValdImageFolders(root=rawtrainfolder, size=size, batch_size=batch_size, num_threads=6, device_id=dali_device)
rawtrainloader = dali.plugin.pytorch.DALIGenericIterator([rawtrainpipe], reader_name ='reader', output_map=['image', 'label'],
                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)
valpipe = DALIValdImageFolders(root=valfolder, size=size, batch_size=batch_size, num_threads=4, device_id=dali_device)
valloader = dali.plugin.pytorch.DALIGenericIterator([valpipe], reader_name='reader', output_map=['image', 'label'],
                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)

In [5]:
cluster = np.zeros((1201, 4))
for i in range(0, 1201, 10):
    t = time.time()
    with T.inference_mode():
        with T.autocast(device_type='cuda', dtype=T.float16, enabled=use_amp):
            model.load_state_dict(scp_Tload('%s/%04d.pth' %(root, i), map_location=device))
            model = model.to(device, memory_format=T.channels_last)
            X, y = extract_feature(rawtrainloader, model, device, normalize=True, use_amp=use_amp)
            cluster[i, 0] = calinski_harabasz_score(X, y)
            cluster[i, 1:] = [k.detach().cpu().numpy() for k in relative_local_density_score(X, y, [1, 0.5, 0.1])]
    scp_Tsave(cluster, '%s/cluster.pth' %root)
    print(i)
    print(cluster[i])
    print(time.time() - t, end='\n\n')

0
[5.52402832e+02 1.28383115e-02 2.77401377e-02 1.26916364e-01]
51.859400033950806

10
[7.82232178e+02 1.66925583e-02 3.37592624e-02 1.02041282e-01]
41.22956848144531

20
[5.21322205e+02 1.36753991e-02 4.54203039e-02 1.32086098e-01]
41.11357235908508

30
[2.60977478e+02 8.81754793e-03 3.67530696e-02 1.29159927e-01]
40.92232370376587

40
[1.59562958e+02 1.32948356e-02 1.21261820e-01 1.87391564e-01]
41.48855519294739

50
[1.07984261e+02 8.52002576e-03 8.57318193e-02 1.88764676e-01]
40.27677321434021

60
[1.27971786e+02 6.38608355e-03 4.70085368e-02 2.08074510e-01]
42.16154217720032

70
[1.42348221e+02 6.78487495e-03 7.06409663e-02 2.37330258e-01]
42.01363444328308

80
[1.14206741e+02 5.33851516e-03 9.85946134e-02 2.36444890e-01]
41.81217050552368

90
[1.04733017e+02 5.10225073e-03 9.12558585e-02 2.41672128e-01]
40.720154762268066

100
[1.06157532e+02 4.76734061e-03 9.01093930e-02 2.46204615e-01]
41.55443859100342

110
[1.15389923e+02 4.14207717e-03 5.26684485e-02 2.74261206e-01]
41.45995